In [1]:
# Install dependencies

#Install all dependency packages for the course
#Remember to execute this before running any of the exercises
!pip install tenacity==9.0.0
!pip install langchain==0.3.12
!pip install langchain-openai==0.2.12
!pip install langchain_community==0.3.12
!pip install langgraph==0.2.59
!pip install pysqlite3-binary==0.5.4
!pip install langchain_chroma==0.1.4
!pip install pandas==2.2.3
!pip install pypdf==5.1.0
!pip install nbformat==5.10.4

  Using cached sqlalchemy-2.0.43-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached multidict-6.6.4-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (73 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a w

In [27]:
#setup models

# from langchain_openai import AzureChatOpenAI
# from langchain_openai import AzureOpenAIEmbeddings
from openai import OpenAI
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

# Load variables from .env into environment
load_dotenv()

# Now you can access the key
api_key = os.getenv("OPENAI_API_KEY")

# Setup the client
client = OpenAI(api_key=api_key)

# # Setup the LLM
response = client.embeddings.create(
    model="text-embedding-3-large",
    input="This is a test sentence."
)

embedding_vector = response.data[0].embedding
# print(embedding_vector)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

# model = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Hello! Can you summarize this text?"}
#     ]
# )

# print(response.choices[0].message.content)


In [28]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("data/Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the pricxe of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
print(get_laptop_price("alpha"))
print(get_laptop_price("testing"))

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2
1499
-1


In [29]:
import sqlite3

conn = sqlite3.connect(":memory:")
print(sqlite3.sqlite_version)


3.50.4


In [30]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma 
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits, 
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
# print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [31]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                state_modifier=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [32]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_Bzztaf9GxkKz6n4vc6hB9Kzg)
 Call ID: call_Bzztaf9GxkKz6n4vc6hB9Kzg
  Args:
    query: GammaAir
  get_laptop_price (call_mRwuf8mqIhUaCjqqaPUNovkf)
 Call ID: call_mRwuf8mqIhUaCjqqaPUNovkf
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The GammaAir X laptop features the following specifications:

- **Processor:** AMD Ryzen 7
- **Memory:** 32GB of DDR4 RAM
- **Storage:** 512GB NVMe SSD
- **Design:** Thin and light form factor, perfect for users who need high performance in a portable design.

The price for the GammaAir X is $1399.


In [33]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today with our range of laptops?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you with that. Do you have any specific features or requirements in mind for your new laptop?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here is a list of our available laptops:

1. **AlphaBook Pro**: A sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM, and a 1TB SSD.

2. **GammaAir X**: Combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB NVMe SSD.

3. **SpectraBook S**: Features an Intel Core i9 processor, 64GB RAM, and a 2TB SSD, ideal for power users.

4. **OmegaPro G17**: A gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB SSD.

5. **NanoEdge Flex**: (Details not provided in the current data)

If you need more information about any specific model or their

In [34]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: The SpectraBook S is designed for power users and features the following specifications:

- **Processor**: Intel Core i9
- **Memory**: 64GB RAM
- **Storage**: 2TB SSD

This workstation-class laptop is perfect for intensive tasks like video editing and 3D rendering.

USER 2: The GammaAir X is equipped with the following features:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4 RAM
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and light form factor, making it perfect for users who need high performance in a portable design.

USER 1: The price of the SpectraBook S is $2,499.

USER 2: The price of the GammaAir X is $1,399.
